# KAG: Knowledge Augmented Generation

 
## Overview

 
KAG is a logical reasoning and Q&A framework based on the OpenSPG engine and large language models, which is used to build logical reasoning and Q&A solutions for vertical domain knowledge bases. KAG can effectively overcome the ambiguity of traditional RAG vector similarity calculation and the noise problem of GraphRAG introduced by OpenIE. KAG supports logical reasoning and multi-hop fact Q&A, etc., and is significantly better than the current SOTA method

## Motivation

 
The goal of KAG is to build a knowledge-enhanced LLM service framework in professional domains, supporting logical reasoning, factual Q&A, etc. KAG fully integrates the logical and factual characteristics of the KGs. Its core features include:

- Knowledge and Chunk Mutual Indexing structure to integrate more complete contextual text information.

- Knowledge alignment using conceptual semantic reasoning to alleviate the noise problem caused by OpenIE.

- Schema-constrained knowledge construction to support the representation and construction of domain expert knowledge.

- Logical form-guided hybrid reasoning and retrieval to support logical reasoning and multi-hop reasoning Q&A.

## Key Components

Knowledge Representation: In the context of private knowledge bases, unstructured data, structured information, and business expert experience often coexist. KAG references the DIKW hierarchy to upgrade SPG to a version that is friendly to LLMs. 
Mixed Reasoning Guided by Logic Forms: KAG proposes a logically formal guided hybrid solution and inference engine. 


KAG ( [openspg.github.io](https://openspg.github.io/v2/) )


<div style="text-align: center;">

<img src="https://github.com/OpenSPG/KAG/raw/master/_static/images/kag-arch.png" alt="adaptive retrieval" style="width:100%; height:auto;">
</div>

The KAG framework includes three parts: kg-builder, kg-solver, and kag-model. This release only involves the first two parts, kag-model will be gradually open source release in the future. 


# Engine & Dependent Image Installation

Use the following commands to download the docker-compose.yml file and launch the services with Docker Compose..


In [ ]:
# set the HOME environment variable (only Windows users need to execute this command)
# set HOME=%USERPROFILE%
curl -sSL https://raw.githubusercontent.com/OpenSPG/openspg/refs/heads/master/dev/release/docker-compose-west.yml -o docker-compose-west.yml
docker compose -f docker-compose-west.yml up -d


# Installation of KAG

Please refer to [KAG usage (developer mode)](https://openspg.github.io/v2/docs_en) guide for detailed introduction of the toolkit. Then you can use the built-in components to reproduce the performance results of the built-in datasets, and apply those components to new busineness scenarios.


In [ ]:
# macOS / Linux developers
Create conda env: conda create -n kag-demo python=3.10 && conda activate kag-demo
Clone code: git clone https://github.com/OpenSPG/KAG.git
Install KAG: cd KAG && pip install -e .


# Windows developers
Install the official Python 3.8.10 or later, install Git.
Create and activate Python venv: py -m venv kag-demo && kag-demo\Scripts\activate 
Clone code: git clone https://github.com/OpenSPG/KAG.git
Install KAG: cd KAG && pip install -e .
